In [ ]:
import numpy as np
from qiskit import Aer, QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit import Parameter, Qubit
from qiskit.providers.fake_provider import FakeQuito

In [ ]:
def calc_expected_fidelity_ibm(qc: QuantumCircuit, calibration) -> float:
    res = 1.0
    for instruction, qargs, _cargs in qc.data:
        gate_type = instruction.name

        assert gate_type in ["rz", "sx", "x", "cx", "measure", "barrier"]

        if gate_type != "barrier":
            assert len(qargs) in [1, 2]
            first_qubit = calc_qubit_index(qargs, qc.qregs, 0)
            if len(qargs) == 1:
                if gate_type == "measure":
                    specific_error: float = calibration.readout_error(first_qubit)
                else:
                    specific_error = calibration.gate_error(gate_type, [first_qubit])

            else:
                second_qubit = calc_qubit_index(qargs, qc.qregs, 1)

                specific_error = calibration.gate_error(gate_type, [first_qubit, second_qubit])

            res *= 1 - specific_error
    return res


def calc_qubit_index(qargs: list[Qubit], qregs: list[QuantumRegister], index: int) -> int:
    offset = 0
    for reg in qregs:
        if qargs[index] not in reg:
            offset += reg.size
        else:
            qubit_index: int = offset + reg.index(qargs[index])
            return qubit_index
    error_msg = f"Global qubit index for local qubit {index} index not found."
    raise ValueError(error_msg)

In [ ]:
basis_gates_ibm = ["rz", "x", "sx", "cx", "id"]
qc = QuantumCircuit(4)

qc.rz(np.pi, 0)
qc.h([0, 1])
qc.cx(0, 2)
qc.cx(1, 3)
qc.rz(Parameter("x"), [0, 1, 2, 3])
qc.cx(0, 1)
qc.cx(2, 3)
qc.measure_all()
qc.draw(fold=-1)

In [ ]:
synthesized_qc = transpile(qc, basis_gates=basis_gates_ibm, optimization_level=3)
synthesized_qc.draw(fold=-1)

In [ ]:
backend = FakeQuito()
mapped_qc = transpile(
    synthesized_qc,
    basis_gates=basis_gates_ibm,
    optimization_level=3,
    coupling_map=backend.configuration().coupling_map,
    seed_transpiler=9,
)
mapped_qc.draw(fold=-1)

In [ ]:
calc_expected_fidelity_ibm(mapped_qc, backend.properties())

In [ ]:
mapped_qc = mapped_qc.assign_parameters({mapped_qc.parameters[0]: np.pi})

In [ ]:
mapped_qc.draw(fold=-1)

In [ ]:
mapped_qc.draw(fold=-1)
backend = FakeQuito()
num_shots = 10000
res_fake = backend.run(mapped_qc, shots=num_shots).result()
counts_fake = res_fake.get_counts(mapped_qc)
counts_fake

In [ ]:
simulator = Aer.get_backend("aer_simulator_statevector")
result = simulator.run(mapped_qc).result()
counts = result.get_counts(mapped_qc)
print(counts)
optimal_counts = {"1010": num_shots / 4, "1111": num_shots / 4, "0000": num_shots / 4, "0101": num_shots / 4}

In [ ]:
def hist_intersection(
    original_counts: dict[str, int],
    current_counts: dict[str, int],
) -> float:
    all_keys = set(original_counts.keys()) | set(current_counts.keys())

    ideal_counts_filled = {key: original_counts.get(key, 0) for key in all_keys}
    counts_noisy_filled = {key: current_counts.get(key, 0) for key in all_keys}

    assert len(ideal_counts_filled.values()) == len(counts_noisy_filled.values())

    tmp_sum = 0
    for i in range(len(ideal_counts_filled.values())):
        tmp_sum += min(list(ideal_counts_filled.values())[i], list(counts_noisy_filled.values())[i])

    return tmp_sum / sum(original_counts.values())

In [ ]:
hist_intersection(optimal_counts, counts_fake)